# Part  1: Scraping

In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import os

## NASA Mars News

In [2]:
# Set up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/coreylawson-enos/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [3]:
# Link to website
url = 'https://redplanetscience.com/#'
browser.visit(url)

In [4]:
# Scrape website and store first news article's title and paragraph in variables
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

HiRISE Views NASA's InSight and Curiosity on Mars
New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.


## JPL Mars Space Images—Featured Image

In [5]:
# Link to website
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [6]:
# Scrape website for featured image URL
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

featured_image_url = 'https://spaceimages-mars.com/' + soup.find('a', class_='showimg fancybox-thumbs')['href']

print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


## Mars Facts